In [ ]:
from collections import deque
from collections import Counter
import random
class Player:
  def __init__(self, name):
    self.name = name
    self.hand = deque()  #Deque for each player cards in hand

  def ask_for_card(self, other_player, rank):
    if rank not in [card.split()[0] for card in self.hand]:
      print(f"{self.name} doesn't have any {rank}.")
      return False
    if rank in [card.split()[0] for card in other_player.hand]:
      card_to_give = next(card for card in other_player.hand if rank in card.split())
      self.hand.append(card_to_give)
      other_player.hand.remove(card_to_give)
      print(f"{other_player.name} gives {card_to_give} to {self.name}")
      return True
    else:
      print(f"{other_player.name} doesn't have any {rank}. Go Fish!")
      return False

  def compare_with_opponents(self, opponents, choice):
    print("Opponent Function", opponents.hand)
    for opponent in opponents.hand:
      for card in self.hand:

        if opponent.split()[0] == card.split()[0] and choice in [card.split()[0] for card in opponents.hand] and choice == opponent.split()[0]:
            print("Opponent ", opponent,opponent.split()[0])
            print("Card", card.split()[0],"-",card)
            print("Choice", choice)
            self.hand.remove(card)
            opponents.hand.remove(opponent)
            return self.hand
            # print(f"{self.name} gave {opponent.name} a {card}")



  def go_fish(self, deck,player,opponents,choice):
    if deck:
      drawn_card = deck.popleft()  # Draw from the left end (top of the deck) from card deck deque
      self.hand.append(drawn_card)
      print(f"{self.name} goes fish and gets {drawn_card}")
    else:
      print("Running out of deck")
      return False

      # print(f"There are no cards left in the deck for {self.name} to go fish.")

  def has_empty_hand(self):
    return not self.hand

  def form_pairs(self):

    list_queue = [i.split()[0] for i in self.hand]
    count_dict = {}
    queue_element = Counter(list_queue)
    for item in self.hand:
      count_dict[item] = queue_element [item.split()[0]]
    formed_pairs = []
    itr = 0
    for item, count in count_dict.items():
      if count == 2:
          print(item)
          pair = self.hand.remove(item)
          formed_pairs.append(item)
      elif count > 2:
          indices = [i for i in range(len(list_queue)) if list_queue[i] == item.split()[0]]
          if len(indices)%2==0:
            pair = self.hand.remove(item)
            formed_pairs.append(item)
          else:
            itr += 1
            if itr < 3:
              pair = self.hand.remove(item)
              formed_pairs.append(item)

    return self.hand, formed_pairs

class Card:
  def __init__(self, rank, suit):
    self.rank = rank
    self.suit = suit

  def __str__(self):
    return f"{self.rank} of {self.suit}"

def create_deck():
  suits = ["Hearts", "Diamonds", "Clubs", "Spades"]
  ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
  ranks = [Card(rank, suit) for rank in ranks for suit in suits]
  deck = deque([f"{rank}" for rank in ranks]) #deque for creating initial card deck
  random.shuffle(deck)  # Randomly Shuffling the deck
  return deck

def play_go_fish(deck, num_players, num_cards_per_player):
  start = "\033[1m"
  end = "\033[0;0m"
  players = [Player(f"Player {i+1}") for i in range(num_players)]
  for player in players:
    for _ in range(num_cards_per_player):
      player.hand.append(deck.popleft())  # Assigning Cards to each player
  player_selection = [i for i in range(0, num_players)]
  current_player = random.choice(player_selection)
  form_pairs = {}
  round_count = 0
  while not any(player.has_empty_hand() for player in players):
    round_count += 1
    print(f"{start}Current Player{end}:", current_player+1)
    print(f"{start}Round Count{end}: {round_count}")
    player = players[current_player]
    player.hand, formed_pairs = player.form_pairs()  # Check and remove pairs

    if formed_pairs:
      if player.name in form_pairs:
        form_pairs[player.name] += 1
      else:
        form_pairs[player.name] = int(len(formed_pairs)/2)
      print(f"{start}{player.name} forms a pair{end}: {', '.join(formed_pairs)}")
    rank_list = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
    # choice = random.choice(rank_list)
    # if any(player.has_empty_hand() for player in players):
    print(f"{start}{player.name}{end}", "----", list(player.hand))
    for count,other_player in enumerate(players):
      # print(count, other_player.name, player.name)
      if player.name != other_player.name:
        other_player.hand, other_player_formed_pairs = other_player.form_pairs()
        if other_player_formed_pairs:
          if other_player.name in form_pairs:
            form_pairs[other_player.name] += 1
          else:
            form_pairs[other_player.name] = int(len(other_player_formed_pairs)/2)
          print(f"{other_player.name} forms a pair: {', '.join(other_player_formed_pairs)}")
        print(f"{start}{other_player.name}{end}", "---", list(other_player.hand))

      #   current_player = (current_player + 1) % num_players
    next_player = (current_player + 1) % num_players + 1
    choice = input(f"Enter Number you wish to ask next player {next_player}: ")
    found = False
    for loop_count, other_player in enumerate(players[(current_player + 1) % num_players:]):
      if player.name != other_player.name and loop_count == 0:
        if player.ask_for_card(other_player, choice):
          found = True
          player.hand, formed_pairs = player.form_pairs()
          if formed_pairs:
            if player.name in form_pairs:
              form_pairs[player.name] += 1
            else:
              form_pairs[player.name] = int(len(formed_pairs)/2)

              break
    if not found:
      opponents = players[(current_player + 1) % num_players:]
      fish_pond = player.go_fish(deck,player,opponents,choice)
      player.hand, formed_pairs = player.form_pairs()
      if formed_pairs:
        if player.name in form_pairs:
          form_pairs[player.name] += 1
        else:
          form_pairs[player.name] = int(len(formed_pairs)/2)
      if fish_pond == False:
        print("Enter deck out of stock")
        for other_player in players:
          if player.name != other_player.name:
            set2 = set(other_player.hand)
            compare = player.compare_with_opponents(other_player,choice)
            if compare != None:
              # print("Pair",player.name, "-", compare)
              if player.name in form_pairs:
                form_pairs[player.name] += 1
              else:
                form_pairs[player.name] = int(len(formed_pairs)/2)

      print(f"{start}After Get Card from Pond{end}", list(player.hand))
    winner_dict = {}
    # Check for winner after each turn

    if any(player.has_empty_hand() for player in players):
      winner = max(form_pairs, key=form_pairs.get)
      print("Final Pair Count:", form_pairs)
      # winner = "".join([player for player in players if len(player.hand)==0 and form_pairs[player.name] == max_pairs])
      print(f"{winner} wins!")

      # winner_dict = {winning_player.name: len(winning_player.hand) for winning_player in players}
      # winner = max(winner_dict, key=winner_dict.get)
      # print(f"{winner} wins!")
    if not found:
      current_player = (current_player + 1) % num_players
    else:
      current_player = current_player

    print(f"{start}Pair Count after Round {round_count}: {end}", form_pairs)
    print(f"{start}Cards Remaining in th Deck {end}: {len(deck)} ")
    print("-"*150)


if __name__ == "__main__":
  game_deck = create_deck()
  player_count = int(input("Enter the No.of.Players: "))
  if player_count == 2:
    num_cards_per_player = 7
  elif player_count ==3 or player_count == 4:
    num_cards_per_player = 5
  else:
    print("Enter Valid player count")

  play_go_fish(game_deck,player_count, num_cards_per_player)


Current Player: 1
Round Count: 1
2 of Hearts
2 of Spades
Player 1 forms a pair: 2 of Hearts, 2 of Spades
Player 1 ---- ['5 of Diamonds', 'A of Hearts', '6 of Clubs']
3 of Diamonds
3 of Spades
Player 2 forms a pair: 3 of Diamonds, 3 of Spades
Player 2 --- ['A of Diamonds', '8 of Hearts', '5 of Clubs']
Player 3 --- ['2 of Diamonds', '4 of Hearts', 'J of Diamonds', '6 of Diamonds', '5 of Spades']
K of Diamonds
K of Spades
Player 4 forms a pair: K of Diamonds, K of Spades
Player 4 --- ['8 of Spades', '9 of Diamonds', '7 of Spades']
Player 1 doesn't have any a.
Player 1 goes fish and gets 8 of Diamonds
After Get Card from Pond ['5 of Diamonds', 'A of Hearts', '6 of Clubs', '8 of Diamonds']
Pair Count after Round 1:  {'Player 1': 1, 'Player 2': 1, 'Player 4': 1}
Cards Remaining in th Deck : 31 
------------------------------------------------------------------------------------------------------------------------------------------------------
Current Player: 2
Round Count: 2
Player 2 ---- ['